# Creating TFRecords

**Author:** [Dimitre Oliveira](https://www.linkedin.com/in/dimitre-oliveira-7a1a0113a/)<br>
**Date created:** 2021/02/27<br>
**Last modified:** 2021/02/27<br>
**Description:** Converting data to the TFRecord format.

## Introduction

The TFRecord format is a simple format for storing a sequence of binary records.
Converting your data into TFRecord has many advantages, such as:

- **More efficient storage**: the TFRecord data can take up less space than the original
data; it can also be partitioned into multiple files.
- **Fast I/O**: the TFRecord format can be read with parallel I/O operations, which is
useful for [TPUs](https://www.tensorflow.org/guide/tpu) or multiple hosts.
- **Self-contained files**: the TFRecord data can be read from a single source—for
example, the [COCO2017](https://cocodataset.org/) dataset originally stores data in
two folders ("images" and "annotations").

An important use case of the TFRecord data format  is training on TPUs. First, TPUs are
fast enough to benefit from optimized I/O operations. In addition, TPUs require
data to be stored remotely (e.g. on Google Cloud Storage) and using the TFRecord format
makes it easier to load the data without batch-downloading.

Performance using the TFRecord format can be further improved if you also use
it with the [tf.data](https://www.tensorflow.org/guide/data) API.

In this example you will learn how to convert data of different types (image, text, and
numeric) into TFRecord.

**Reference**

- [TFRecord and tf.train.Example](https://www.tensorflow.org/tutorials/load_data/tfrecord)


## Dependencies

In [1]:
import os
import json
import pprint
import tensorflow as tf
import matplotlib.pyplot as plt

## Download the COCO2017 dataset

We will be using the [COCO2017](https://cocodataset.org/) dataset, because it has many
different types of features, including images, floating point data, and lists.
It will serve as a good example of how to encode different features into the TFRecord
format.

This dataset has two sets of fields: images and annotation meta-data.

The images are a collection of JPG files and the meta-data are stored in a JSON file
which, according to the [official site](https://cocodataset.org/#format-data),
contains the following properties:

```
id: int,
image_id: int,
category_id: int,
segmentation: RLE or [polygon], object segmentation mask
bbox: [x,y,width,height], object bounding box coordinates
area: float, area of the bounding box
iscrowd: 0 or 1, is single object or a collection
```

In [2]:
root_dir = "datasets"
tfrecords_dir = "tfrecords"
train_images_dir = os.path.join(root_dir, "train2017")
val_images_dir = os.path.join(root_dir, "val2017")

annotations_dir = os.path.join(root_dir, "annotations")
train_annotation_file = os.path.join(annotations_dir, "instances_train2017.json")
val_annotation_file = os.path.join(annotations_dir, "instances_val2017.json")

train_captions_annotation_file = os.path.join(annotations_dir, "captions_train2017.json")
val_captions_annotation_file = os.path.join(annotations_dir, "captions_val2017.json")

train_images_url = "http://images.cocodataset.org/zips/train2017.zip"
val_images_url = "http://images.cocodataset.org/zips/val2017.zip"
annotations_url = (
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
)

# Download image files
if not os.path.exists(val_images_dir):
    val_image_zip = tf.keras.utils.get_file(
        "images.zip", cache_dir=os.path.abspath("."), origin=val_images_url, extract=True,
    )
    os.remove(val_image_zip)

# Download image files
if not os.path.exists(train_images_dir):
    train_image_zip = tf.keras.utils.get_file(
        "train_images.zip", cache_dir=os.path.abspath("."), origin=train_images_url, extract=True,
    )
    os.remove(train_image_zip)

# Download caption annotation files
if not os.path.exists(annotations_dir):
    annotation_zip = tf.keras.utils.get_file(
        "captions.zip",
        cache_dir=os.path.abspath("."),
        origin=annotations_url,
        extract=True,
    )
    os.remove(annotation_zip)

    
print("The COCO dataset has been downloaded and extracted successfully.")

with open(train_annotation_file, "r") as f:
    train_annotations = json.load(f)["annotations"]

with open(val_annotation_file, "r") as f:
    val_annotations = json.load(f)["annotations"]

with open(train_captions_annotation_file, "r") as f:
    train_captions_annotations = json.load(f)["annotations"]

with open(val_captions_annotation_file, "r") as f:
    val_captions_annotations = json.load(f)["annotations"]
    
print(f"Number of train images: {len(train_annotations)}")
print(f"Number of val images: {len(val_annotations)}")


The COCO dataset has been downloaded and extracted successfully.
Number of train images: 860001
Number of val images: 36781


### Contents of the COCO2017 dataset

In [3]:
# pprint.pprint(train_captions_annotations[0])
def getCaptions(captions_annotations,image_id):
    str = ""
    for a in train_captions_annotations:
        if(a['image_id'] == 203564):
            str += '<start> ' + a['caption'].rstrip() + ' <end>' + '\n'
    return str
str = getCaptions(train_captions_annotations, 203564)
print(str)
# train_captions_annotations[]
# train_captions_annotations

<start> A bicycle replica with a clock as the front wheel. <end>
<start> The bike has a clock as a tire. <end>
<start> A black metal bicycle with a clock inside the front wheel. <end>
<start> A bicycle figurine in which the front wheel is replaced with a clock <end>
<start> A clock with the appearance of the wheel of a bicycle <end>



## Parameters

`num_samples` is the number of data samples on each TFRecord file.

`num_tfrecods` is total number of TFRecords that we will create.

In [4]:
num_samples = 4096
num_train_tfrecods = len(train_annotations) // num_samples
num_val_tfrecods = len(val_annotations) // num_samples
if len(train_annotations) % num_samples:
    num_train_tfrecods += 1  # add one record if there are any remaining samples
if len(val_annotations) % num_samples:
    num_val_tfrecods += 1  # add one record if there are any remaining samples

if not os.path.exists(tfrecords_dir):
    os.makedirs(tfrecords_dir)  # creating TFRecords output folder

In [5]:
split_layer = 100
saved_model_path = '/home/suphale/WorkSpace/saved_model'
model_path = saved_model_path + '/iv3_head_model_%d' % (split_layer)
head_model = tf.keras.models.load_model(model_path, compile=False)

model_path = saved_model_path + '/iv3_tail_model_%d' % (split_layer)
tail_model = tf.keras.models.load_model(model_path, compile=False)

2021-07-31 13:13:34.896629: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-31 13:13:34.896884: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-31 13:13:34.898336: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Define TFRecords helper functions

In [6]:

def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
    )

def tensor_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[value.numpy().tobytes()])
    )

def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))


def float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def float_feature_list(value):
    """Returns a list of float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def create_example(image, path, example,captions, h):
    feature = {
        "image": image_feature(image),
        "tensor": tensor_feature(h),
        "path": bytes_feature(path),
        "area": float_feature(example["area"]),
        "bbox": float_feature_list(example["bbox"]),
        "category_id": int64_feature(example["category_id"]),
        "id": int64_feature(example["id"]),
        "image_id": int64_feature(example["image_id"]),
        "captions": bytes_feature(captions),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


def parse_tfrecord_fn(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "tensor": tf.io.FixedLenFeature([], tf.string),
        "path": tf.io.FixedLenFeature([], tf.string),
        "area": tf.io.FixedLenFeature([], tf.float32),
        "bbox": tf.io.VarLenFeature(tf.float32),
        "category_id": tf.io.FixedLenFeature([], tf.int64),
        "id": tf.io.FixedLenFeature([], tf.int64),
        "image_id": tf.io.FixedLenFeature([], tf.int64),
        "captions": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
    example["bbox"] = tf.sparse.to_dense(example["bbox"])
    return example


## Generate data in the TFRecord format

Let's generate the COCO2017 data in the TFRecord format. The format will be
`file_{number}.tfrec` (this is optional, but including the number sequences in the file
names can make counting easier).

In [7]:
# num_tfrecods

In [10]:
from tqdm import tqdm

# num_tfrecods = 1
num_train_tfrecods = 50
# num_val_tfrecods = 1
print("num_train_tfrecods=%d" % num_train_tfrecods)
print("num_val_tfrecods=%d" % num_val_tfrecods)

if(False):
    for tfrec_num in range(num_train_tfrecods):
        samples = train_annotations[(tfrec_num * num_samples) : ((tfrec_num + 1) * num_samples)]
        options = tf.io.TFRecordOptions(compression_type="GZIP")
        with tf.io.TFRecordWriter(
            tfrecords_dir + "/file_train_%.2i-%i.tfrec" % (tfrec_num, len(samples)),options=options
        ) as writer:
            for sample in tqdm(samples):
                image_path = f"{train_images_dir}/{sample['image_id']:012d}.jpg"
                image = tf.io.decode_jpeg(tf.io.read_file(image_path), channels=3)
                captions = getCaptions(train_captions_annotations, sample['image_id'])
                batch_image = tf.image.resize(image, (250, 250))
                batch_image = tf.expand_dims(batch_image, 0) 
                h = head_model(batch_image)
                example = create_example(image, image_path, sample,captions, h)
                writer.write(example.SerializeToString())
                
if(True):
    for tfrec_num in range(num_val_tfrecods):
        samples = val_annotations[(tfrec_num * num_samples) : ((tfrec_num + 1) * num_samples)]
        options = tf.io.TFRecordOptions(compression_type="GZIP")

        with tf.io.TFRecordWriter(
            tfrecords_dir + "/file_val_%.2i-%i.tfrec" % (tfrec_num, len(samples)),options=options
        ) as writer:
            for sample in tqdm(samples):
                image_path = f"{val_images_dir}/{sample['image_id']:012d}.jpg"
                image = tf.io.decode_jpeg(tf.io.read_file(image_path), channels=3)
                captions = getCaptions(val_captions_annotations, sample['image_id'])
                batch_image = tf.image.resize(image, (250, 250))
                batch_image = tf.expand_dims(batch_image, 0) 
                h = head_model(batch_image)
                example = create_example(image, image_path, sample,captions,h)
                writer.write(example.SerializeToString())
                

num_train_tfrecods=50
num_val_tfrecods=9


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4013/4013 [07:30<00:00,  8.91it/s]


In [11]:
image = tf.io.decode_jpeg(tf.io.read_file('./datasets/val2017/000000403353.jpg'),channels=3)
image = tf.image.resize(image, (250, 250))
tf.shape(image)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([250, 250,   3], dtype=int32)>

In [12]:
batch_image = tf.expand_dims(image, 0) 
h = head_model(batch_image)
tf.shape(h)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([  1,  13,  13, 768], dtype=int32)>

In [13]:
# b = tf.io.encode_jpeg(image).numpy()
# type(b)

In [14]:
import numpy as np
from   numpy import float32
from   numpy import byte

print(tf.shape(h))
bytes_list=h.numpy().tobytes()
print(type(bytes_list))
# h_new = tf.reshape(bytes_list,[  1  ,13,  13, 768])
# b = tf.io.decode_raw(bytes_list, tf.int32)
# h_tensor = tf.Tensor(bytes_list)
# h_new = tf.reshape(b,[  1  ,28,  38, 768])
# print(tf.shape(h_new))

generated_np_array = np.frombuffer(bytes_list, dtype=np.int32)
# generated_np_array = np.frombuffer(generated_np_array, dtype=np.int32)
generated_image_np_array = generated_np_array.reshape([  1  ,13,  13, 768])
image_tensor = tf.convert_to_tensor(generated_image_np_array, dtype=tf.int32)
print(tf.shape(h))
print(tf.shape(image_tensor))

tf.Tensor([  1  13  13 768], shape=(4,), dtype=int32)
<class 'bytes'>
tf.Tensor([  1  13  13 768], shape=(4,), dtype=int32)
tf.Tensor([  1  13  13 768], shape=(4,), dtype=int32)


## Explore one sample from the generated TFRecord

In [15]:
print(num_samples)
train_raw_dataset = tf.data.TFRecordDataset(f"{tfrecords_dir}/file_train_00-{num_samples}.tfrec",compression_type='GZIP')
train_parsed_dataset = train_raw_dataset.map(parse_tfrecord_fn)
test_bytes = None
for features in train_parsed_dataset.take(1):
    for key in features.keys():
        if key != "image" and key != "tensor":
            print(f"{key}: {features[key]}")

    print(f"Image shape: {features['image'].shape}")
    print(f"Tensor shape: {features['tensor'].shape}")
    print(tf.shape(features["tensor"].numpy()))
    test_bytes = features["tensor"]
    print(type(test_bytes))
    bytes_list = features["tensor"].numpy()
    print(len(bytes_list))
    generated_np_array = np.frombuffer(bytes_list, dtype=np.int32)
    generated_image_np_array = generated_np_array.reshape([  1  ,13,  13, 768])
    image_tensor = tf.convert_to_tensor(generated_image_np_array, dtype=tf.int32)

#     h_new = tf.reshape(bytes_list,[  1  ,28,  38, 768])
    
#     h_tensor = tf.Tensor(bytes_list)
#     h_new = tf.reshape(bytes_list,[  1  ,28,  38, 768])

#     plt.figure(figsize=(7, 7))
#     plt.imshow(features["image"].numpy())
#     plt.show()

4096
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
bbox: [199.84 200.46  77.71  70.88]
area: 2765.148681640625
captions: b'<start> A bicycle replica with a clock as the front wheel. <end>\n<start> The bike has a clock as a tire. <end>\n<start> A black metal bicycle with a clock inside the front wheel. <end>\n<start> A bicycle figurine in which the front wheel is replaced with a clock <end>\n<start> A clock with the appearance of the wheel of 

2021-07-31 20:56:54.873322: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-07-31 20:56:54.873813: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2800210000 Hz


In [16]:
type(test_bytes)

tensorflow.python.framework.ops.EagerTensor

In [17]:
val_raw_dataset = tf.data.TFRecordDataset(f"{tfrecords_dir}/file_train_00-{num_samples}.tfrec",compression_type='GZIP')
val_parsed_dataset = val_raw_dataset.map(parse_tfrecord_fn)

for features in val_parsed_dataset.take(1):
    for key in features.keys():
        if key != "image" and key != "tensor":
            print(f"{key}: {features[key]}")

#     print(f"Image shape: {features['image'].shape}")
    print(f"Tensor shape: {features['tensor'].shape}")
#     print(len(features["tensor"].numpy()))
#     print(type(features["tensor"]))
#     h = 
    
#     new_tensor = tf.reshape(features["tensor"].numpy(),[  1  ,28,  38, 768])
#     plt.figure(figsize=(7, 7))
#     plt.imshow(features["image"].numpy())
#     plt.show()

bbox: [199.84 200.46  77.71  70.88]
area: 2765.148681640625
captions: b'<start> A bicycle replica with a clock as the front wheel. <end>\n<start> The bike has a clock as a tire. <end>\n<start> A black metal bicycle with a clock inside the front wheel. <end>\n<start> A bicycle figurine in which the front wheel is replaced with a clock <end>\n<start> A clock with the appearance of the wheel of a bicycle <end>\n'
category_id: 58
id: 156
image_id: 558840
path: b'datasets/train2017/000000558840.jpg'
Tensor shape: ()


## Train a simple model using the generated TFRecords

Another advantage of TFRecord is that you are able to add many features to it and later
use only a few of them, in this case, we are going to use only `image` and `category_id`.

## Define dataset helper functions

In [27]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

# from sklearn.metrics import accuracy_score, precision_score, recall_score
# from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

latent_dim = 10 * 1024 

class Autoencoder(tf.keras.Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(13 * 13 * 768 , activation='sigmoid'),
      layers.Reshape((13, 13, 768))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())


In [28]:
input = layers.Input(shape=(13, 13, 768))

# Encoder
x = layers.Conv2D(256, (3, 3), activation="relu")(input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(256, (2, 2), activation="relu")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)

# Decoder
x = layers.Conv2DTranspose(256, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(256, (3, 3), strides=2, activation="relu")(x)
x = layers.Conv2D(768, (3, 3), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 13, 13, 768)]     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 256)       1769728   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 256)         262400    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 6, 6, 256)         590080    
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 13, 13, 256)       5900

In [30]:
@tf.autograph.experimental.do_not_convert
def prepare_sample(features):
    bytes_list = features["tensor"]
    dr = tf.io.decode_raw(bytes_list, tf.int32)
    image_tensor = tf.reshape(dr, [  13,  13, 768])
    return image_tensor, image_tensor


def get_dataset(filenames, batch_size):
    dataset = (
        tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE,compression_type='GZIP')
        .map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
        .map(prepare_sample, num_parallel_calls=AUTOTUNE)
        .shuffle(batch_size * 10)
        .batch(batch_size)
        .prefetch(AUTOTUNE)
    )
    return dataset


train_filenames = tf.io.gfile.glob(f"{tfrecords_dir}/file_train_0*.tfrec")
val_filenames = tf.io.gfile.glob(f"{tfrecords_dir}/file_val_0*.tfrec")
batch_size = 32
epochs = 10
steps_per_epoch = 50
AUTOTUNE = tf.data.experimental.AUTOTUNE

input_tensor = tf.keras.layers.Input(shape=(224, 224, 3), name="image")
model = tf.keras.applications.EfficientNetB0(
    input_tensor=input_tensor, weights=None, classes=91
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

autoencoder.fit(
    x=get_dataset(train_filenames, batch_size),
    validation_data=(get_dataset(val_filenames, batch_size)),
    epochs=epochs,
#     steps_per_epoch=steps_per_epoch,
    verbose=1,
)

Epoch 1/10


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

## Conclusion

This example demonstrates that instead of reading images and annotations from different
sources you can have your data coming from a single source thanks to TFRecord.
This process can make storing and reading data simpler and more efficient.
For more information, you can go to the [TFRecord and
tf.train.Example](https://www.tensorflow.org/tutorials/load_data/tfrecord) tutorial.

In [22]:
autoencoder.save_weights("auto")

In [23]:
autoencoder.save_weights("./temp/autoencoder.weights")

In [24]:
autoencoder.save("./temp/auto")

2021-08-01 00:10:02.695052: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./temp/auto/assets


In [25]:
# auto = tf.keras.
auto = tf.keras.models.load_model("./temp/auto")